# RAG - Retrieval Augmented Generation


## Опиши архитектуру RAG?

![](https://blog.deepschool.ru/wp-content/uploads/2025/07/rag-vanila-schema1-page-2.drawio-6-page-2.drawio-1024x637.png)

**RAG** — это подход, при котором для запроса к LLM из базы данных извлекается релевантная информация и добавляется к промпту пользователя, чтобы сгенерировать ответ с опорой только на эти данные.

- **Чистка и форматирование** — убираем лишний текст, добавляем недостающий, правим формат.
- **Чанкирование** — разбиваем текст на удобные фрагменты (чанки).
- **Эмбедер** — отображает чанки в векторы (эмбеддинги) для оценки схожести двух текстов не по словам, а через сравнение их векторов (например, с помощью оценки косинусного расстояния).
- **Векторная база данных** — хранит эмбеддинги (опять же, предполагаем, что пока используем только векторный поиск) и соответствующие им чанки.
- **Ретривер (поиск)** — ищет нужные чанки по вопросу пользователя.
- **Реранкер (ранжирование)** — упорядочивает найденные чанки по релевантности и отбрасывает лишние.
- **LLM** — генерирует ответ, используя найденную информацию.
- **Бенчмарки и тесты** — проверяют, насколько хорошо работает система.


**Чистка и форматирование**

Чистота данных напрямую влияет на результаты поиска по базе и на качество генерации ответа. В машинном обучении чистые данные — обязательное условие успеха. RAG-системы — не исключение.

Но если вы собираете первую версию — не стыдитесь пропустить этот пункт 🙂

Сделайте первую версию на скорую руку, чтобы получить удовольствие и какой-то бейзлайн по метрикам, а уже потом улучшайте, замеряя результаты на тестах.

Что нужно сделать в первую очередь:

- удалить неинформативные элементы: навигационные панели, меню, рекламные вставки; например, если вы парсите статьи с сайта, убирайте всё, кроме заголовка и текста;
- исправить орфографические ошибки: используйте инструменты по типу LanguageTool или Yandex Speller для автоматической корректировки орфографических ошибок;
- стандартизировать текст: приводите данные к одному формату (например, удалите лишние пробелы, унифицируйте даты и числовые форматы). Если где-то написано «1 декабря 2010 года», а где-то — «01.12.10», лучше исправить это на единый формат.

Если вы уже сделали пункты выше, но качество поиска и генерации ответов вас не удовлетворило, можно попробовать следующее:

- добавить связи между документами для большего контекста в промпт. Например, если у вас система поиска по задачам в трекере — можно прилинковать описание проекта;
- добавить выделенный список ключевых слов к чанку: это поможет при гибридном поиске (поговорим об этом чуть позже).

**Чанкирование**

Чанкирование — первый этап, на котором текст разбивают на подходящие фрагменты (чанки).

Разбить текст можно тремя алгоритмами:

- По длине чанка — самый простой и популярный вариант. При нём мы рискуем неудачно обрезать важную информацию: половина будет в одном чанке, половина — в другом. При этом во время поиска один из полезных чанков может не извлечься, и ответ LLM будет неправильным.
- По структуре документа — тоже простой алгоритм, немного снижающий вероятность разбить важную информацию на несколько чанков, но не всегда применимый из-за отсутствия структуры.
- По смыслу — более сложный вариант, требующий LLM и/или ручную разметку. Применим, когда есть необходимость, деньги и время улучшать качество.

**Эмбеддер**

Эмбеддер отображает чанки и запросы пользователей в векторы таким образом, чтобы похожие по смыслу тексты были близки друг к другу в векторном пространстве, а далёкие, соответственно, были далеки.

Это и позволяет нам совершать векторный поиск: по запросу пользователя находить самые близкие по смыслу чанки и добавлять их в промпт к модели для получения ответа.

На старте можно смело использовать готовые модели, они часто показывают хорошие результаты, например:

BGE-M3 — популярная многозадачная модель для retrieval и rerank-задач, которая поддерживает несколько языков;
text-embedding-3-large — решение от компании OpenAI;
FRIDA — модель, хорошо работающая на русском языке.

Но если у вас специфическая доменная область — модель «из коробки» может плохо располагать ваши тексты в векторном пространстве.

**База данных**

Большая часть RAG-систем построена на векторном поиске, поэтому пока мы предполагаем, что используем именно её.

После того, как мы получили эмбеддинги для всех чанков, нужно сохранить векторы и соответствующие им тексты в векторной базе данных. Векторная база отличается от обычной тем, что индексом является вектор, и во время поиска мы ищем не точное совпадение, а максимально близкие (например, в понимании косинусного расстояния) векторы.

Если чанков мало (например, пара тысяч), можно обойтись локальным решением (in-memory списком или FAISS), потому что можно «в лоб» сравнить вектор запроса со всеми векторами.

Но если чанков становится много (сотни тысяч и более), такой перебор начинает заметно задерживать ответ. Тут на помощь приходит Approximate Nearest Neighbor (ANN) — подход, позволяющий быстро искать похожие вектора на запрашиваемый, в огромных массивах данных. В этом подходе нет гарантии, что найдутся самые близкие вектора, но точно похожие.

Другой вариант — использовать FAISS, в котором можно делать как и полный перебор подобно алгоритму выше, так и более быстрый поиск за счёт предварительной работы с векторами: построения дерева или кластеризации.

Помимо поиска по векторам часто применяют гибридный поиск: векторный + текстовый. Например, когда база большая, можно выбрать первую группу кандидатов текстовым поиском, а потом отбросить менее релевантные векторным. Также текстовый поиск необходим, когда надо учитывать редкие термины или точные ключевые слова. При этом текстовый поиск — отдельная объёмная тема на несколько статей. Но если надо с чего-то начать, хорошая точка входа — алгоритм BM25.

**Ретривер**

**Ретривер (Retriever)**— это ключевой компонент RAG-системы (Retrieval-Augmented Generation), отвечающий за поиск и выбор наиболее релевантных фрагментов данных из внешней базы знаний для последующей генерации ответа языковой моделью (LLM). Его задача — минимизировать «галлюцинации» LLM и повысить точность ответов, предоставляя модели контекст, основанный на фактах из вашей базы данных.

Что надо сделать на этом шаге:

- **Переформулировать запрос:** пользователи пишут в свободной форме, и потому запрос может быть расплывчатым, содержать ошибки, лишние слова или не подходить под формат данных (храним инструкции, а запрос не похож на текст инструкций). Поэтому зачастую запрос лучше переформулировать. Можно сделать это с помощью LLM, написав промпт: «Переформулируй запрос в форме короткого заголовка справочной статьи, чтобы по нему можно было найти документ в базе знаний». В этом пункте большой простор для экспериментов. Главное помните, что запрос должен быть похож на текст в чанках.
- **Прогнать запрос через эмбеддер:** тут всё просто, но стоит прочитать документацию к эмбеддеру. Например, e5-large-instruct требует описания задачи, иначе работает хуже. А FRIDA — правильных префиксов.
- **Найти ближайшие векторы в базе:** обычно хватает 3-10 кандидатов, но количество зависит от домена и задачи. Иногда уместно варьировать количество кандидатов в зависимости от запроса.

- **Дополнительно обработать результат:** например, можно удалить дубликаты или слишком короткие чанки.


**Реранкер**
В ретривере вы обычно выбираете топ-N записей, но среди них могут быть как актуальные, так и нет. Например, если пользователь спросит про возврат денег, ретривер сможет вернуть и политику возврата средств, и раздел про возврат подарочных сертификатов. Оба фрагмента будут близки по вектору, но второй бесполезен и его надо отбросить.

Также полученные чанки необходимо отсортировать от более к менее полезным. Так делают, потому что у моделей есть тенденция обращать внимание именно на начало инструкций.

Фильтрацию по смыслу с ранжированием и делает реранкер. И да, кажется, должно хватить информации от ретривера: мы же знаем, насколько каждый чанк близок в векторном пространстве к запросу — почему бы не отфильтровать и не упорядочить чанки по косинусному расстоянию? Ответ: просто с реранкером работает лучше 🙂

В первой версии RAG-системы можно обойтись и без реранкера — просто используйте топ-N кандидатов, отсортированных по сходству эмбеддингов и отсекайте кандидатов по порогу.

Если вы хотите повысить качество — можно добавить реранкер на основе LLM или даже специальную модель-классификатор. В обоих случаях надо подать на вход пару «запрос + чанк» и получить оценку релевантности (например, от 0 до 1)

Существуют публичные модели реранкеров. Например:

- bge-reranker — серия моделей от BAAI (есть версии для разных языков, в том числе мультиязычные). Они выложены в открытый доступ, например, на Hugging Face.
- colbert — модель, где поиск и ранжирование делаются с учётом токенов внутри чанков, а не только глобального эмбеддинга. Более продвинутый вариант, но требует настройки.

Более продвинутый вариант — дообучить публичную модель на собственных данных. Обычно это делается как pairwise learning:

в каждом обучающем примере два чанка и запрос;
модель учится выбирать, какой из чанков подходит лучше.
Для обучения можно использовать open-source библиотеки (например, Sentence-Transformers), но на старте это избыточно.

**LLM**

Теперь надо переписать найденные чанки в цельный, логичный и удобный для чтения ответ. Для этого в промпт к модели необходимо вставить информацию из чанков и запрос пользователя.

Начать можно с инструкции:

```
Ответь на вопрос, используя только предоставленные ниже фрагменты. 
Если информации недостаточно, напиши: "Не знаю".

Вопрос: {user_question}

Фрагменты:
{chunk_1}
{chunk_2}
{chunk_3}

Ответ:
```

И далее поиграйте с промптом, чтобы ответ был в нужном стиле, необходимой длины и в идеале без галлюцинаций (шанс появления которых никогда не равен нулю, но можно его минимизировать).

Чтобы снизить галлюцинации:

- добавьте идентификаторы чанкам и инструкцию, чтобы LLM выводила идентификатор фрагмента, на основе которого она сгенерировала эту часть ответа; на постобработке можно их убрать, если не хотите показывать пользователю;
- уменьшите температуру и top-k (или top-p), чтобы сделать ответы более детерменированными;
- добавьте ещё одну модель, которая перепроверит, действительно ли ответ использует только информацию из чанков;
и в последнюю очередь дообучите LLM.


## Какие есть виды поиска в RAG?

В RAG-системах (Retrieval-Augmented Generation) используется несколько типов поиска, которые отличаются тем, **как они ищут релевантные документы** и **как комбинируют сигналы** (лексические, семантические, статистические).

Ниже — полный перечень современных подходов с краткими объяснениями.

---

✅ **1. Лексический поиск (Keyword / Sparse Retrieval)**

Поиск по ключевым словам, работает на разреженных (sparse) векторах.

Основные методы:

* **BM25**
* **TF-IDF**
* **Okapi**
* **Language Models for IR (Classic)**

Плюсы:

* Хорош при точных совпадениях.
* Быстрый, дешёвый.
* Отлично ловит **имена**, **цифры**, **специфические термины**.

Минусы:

* Не понимает семантические синонимы:
  “какая скорость” ≠ “максимальный rate”.

Используется почти в каждой RAG-системе как baseline.

---

✅ **2. Векторный поиск (Dense Retrieval)**

Поиск по эмбеддингам.
Документы и запрос преобразуются в вектора — сравниваются через cosine / dot.

Основные модели:

* **Sentence-BERT / SBERT**
* **E5 / BGE / GTE**
* **OpenAI Embeddings** (text-embedding-3-large)
* **Voyage**, **Cohere**, **Jina**

Плюсы:

* Понимает смысл текста.
* Находит документы, даже если слова разные.

Минусы:

* Может пропускать точные совпадения.
* Нужна векторная БД (FAISS, Milvus, pgvector, Weaviate, Pinecone).

Используется как основной метод в 90% RAG.

---

✅ **3. Гибридный поиск (Hybrid Retrieval)**

Комбинация sparse + dense.
Лучший современный вариант.

Подходы:

* BM25 + Embeddings (score fusion)
* Reciprocal Rank Fusion (RRF)
* Weighted Hybrid (α * dense + β * sparse)
* Cross-encoder reranker после обеих выборок

Плюсы:

* Ловит и точные совпадения, и смысл.
* Сильнейший recall.
* Лучший метод для production RAG.

---

 ✅ **4. Reranking (Relevance Re-ranking)**

Переоценка результатов моделью-кросс-энкодером.

 Популярные reranker-модели:

* **cross-encoder/ms-marco-MiniLM-L-6-v2**
* **BGE-Reranker**
* **Cohere Rerank**
* **ColBERT v2** (partial reranking)
* **DeepImpact, uniCOIL**

 Как работает:

1. Top-k документов выбираются **BM25 / dense / hybrid**.
2. Reranker получает пару:
   `query + candidate_document`
3. Модель точнее определяет релевантность.

Используется в проде, увеличивает качество на 30–60%.

---

 ✅ **5. Мультимодальный поиск (VLM-based Retrieval)**

Если документы — изображения, видео, PDF, UI скриншоты.

Используются:

* CLIP
* SigLIP
* Florence-2
* Llava-Next
* Kosmos-2

Документ и запрос переводятся в общее мультимодальное пространство.

---

✅ **6. Graph-based Retrieval (графовый поиск)**

Используется там, где есть связи между документами.

Методы:

* Knowledge Graph Retrieval
* GraphRAG (от Microsoft)
* Semantic Graph Traversal

Плюсы:

* Отлично работает для сложных структур (manuals, long docs).
* Хорош для reasoning и объяснений.

Минусы:

* Стоимость построения графа.

GraphRAG стал стандартом для enterprise-RAG.

---

🔥 **10. Agentic Retrieval (LLM становится retrieval agent)**

LLM сам решает:

* каким методом искать
* где искать
* каким фильтрам следовать
* сколько итераций сделать

Примеры:

* GPT o1
* ReAct agents
* OpenAI "retrieval-augmented agents"
* DSPy Self-RAG

Это самый мощный подход 2025 года.

---

🚀 Сводная таблица

| Метод             | Понимание смысла | Ловит точные матчи | Лучшее применение      |
| ----------------- | ---------------- | ------------------ | ---------------------- |
| BM25              | ❌                | ✅                  | Когда запросы точные   |
| Dense             | ✅                | ❌                  | Семантический поиск    |
| Hybrid            | ✅                | ✅                  | Production RAG         |
| Reranker          | 🔥               | 🔥                 | Максимальное качество  |
| GraphRAG          | 🧠               | ◑                  | Large knowledge bases  |
| Multi-hop         | 🧠               | ◑                  | Reasoning              |
| Agentic retrieval | 🔥🔥             | 🔥                 | Продвинутые ассистенты |

---




## Какие основные способы улучшить работу RAG системы?

-Дообучить эмбеддер под задачу
...

## Ваша RAG-система начала "галлюцинировать" в продакшене. Как вы проверите, что сломалось — retriever или generator?

Многие кандидаты наверное скажут: «проверить точность» или «запустить больше тестов». Возможно, так и получится найти проблему, но можно пойти чуть иначе.

RAG-системы дают сбой на разных этапах, и для каждого нужны свои метрики. Общая «точность» часто не отвечает на самый важный вопрос — "А где же именно кроется ошибка?"

Ключевая идея:
Качество RAG = Производительность Retriever'а × Производительность Generator'а

Метрики Retrieval (Достали ли мы правильный контекст?)
- Contextual Relevancy: Какой процент полученных чанков действительно релевантен?
- Contextual Recall: Достали ли мы всю необходимую информацию?
- Contextual Precision: Ранжируются ли релевантные чанки выше нерелевантных?

Метрики Generation (Правильно ли LLM использовала контекст?)
- Faithfulness: Насколько вывод соответствует предоставленным фактам?
- Answer Relevancy: Отвечает ли ответ на заданный вопрос?
- Кастомные метрики: Следует ли ответ нужному формату или стилю?

Диагностическая структура:
1️⃣ Высокий Faithfulness + Низкий Relevancy → Проблема в Retrieval
2️⃣ Низкий Faithfulness + Высокий Relevancy → Проблема в Generation
3️⃣  Обе метрики низкие → Сломан весь пайплайн
4️⃣  Обе метрики высокие → Ищите edge-кейсы

Метрика, которая ловит большинство продакшен-проблем: Contextual Recall.
Ваш retriever может находить «релевантный» контент, но упускать критически важные детали. Идеальная точность при нулевой полноте = уверенные, но неправильные ответы. Именно поэтому RAG-системы так уверенно «галлюцинируют».

Но интервьюер может продолжить вас спрашивать:
«У вашего RAG'а точность 85%. А какой accuracy у контекста? Каков score достоверности? Вы меряете end-to-end или на уровне компонентов?»
Если ваши метрики расплывчаты, интервьюер скорее всего решит, что вы не понимаете, как работают RAG-системы в продакшене.

Подход к оценке, который отличает джунов от сеньоров:

Джун: Тестирует всё end-to-end и надеется, что сработает.
Сеньор: Внедряет метрики на уровне компонентов, автоматизированную оценку в CI/CD и мониторинг в продакшене.

Суровая реальность продакшена:
➖ Идеальный retrieval + слабые промты = галлюцинации
➖ Идеальная LLM + плохие чанки = нерелевантные ответы
➖ Хороший retrieval + хорошая генерация + отсутствие мониторинга = неминуемый провал

❗Совет:
Упомяните оценку по методу LLM-as-a-judge.
«Я бы использовал GPT-4 для оценки faithfulness, сравнивая сгенерированные ответы с полученным контекстом, а затем отслеживал распределение скоров over time, чтобы поймать дрейф.»
Это покажет, что вы в курсе современных методов оценки.

Вопрос, который завершает интервью:
«Как бы вы реализовали такую оценку в продакшене?»
Возможный ответ:
- Автоматизированные оценки компонентов в CI/CD
- Мониторинг в реальном времени с оповещениями
- Асинхронная батч-оценка продакшен-трафика

Понимание причин сбоев RAG > заучивание архитектур трансформеров.

## Как перевести в общее мультимодальное пространство RAG для текста и картинок?

## Как выбрать правильную LLM для RAG?

*Полезная литература*


https://blog.deepschool.ru/llm/rag-ot-pervoj-versii-k-rabochemu-resheniyu/?utm_source=tg&utm_medium=post&utm_campaign=blog-article&utm_content=deepschool&utm_term=rag-from-the-first-version-to-a-working-solution

## Как оценивать RAG-систему?

## Метрики RAG

*context relevance* — по вопросу и найденным чанкам проверяем, что нашли что-то нужное;
*context recall* — проверяем, что нашли всё для ответа на вопрос;
*faithfullness* — проверяем, что LLM генерирует ответ, основываясь на данных в промпте, а не выдумывает факты.

## Какие есть проблемы у RAG?

На первый взгляд кажется просто, но на практике первая версия RAG никогда (!) не работает так, как ожидают. Потому что есть много мест, где можно ошибиться:

- использовали плохой эмбеддер (модель, которая отображает текст в вектор), чтобы быстрее искать похожие по смыслу тексты, сравнивая векторы, а не слова (на самом деле не всегда применяется именно векторный поиск, но пока предполагаем именно его);
- неверно сформулировали поисковый запрос, в таком случае использовать текст пользователя в качестве запроса — приемлемое решение, но можно улучшить результаты поиска, переформулировав вопрос;
- неудачно разбили данные на кусочки (чанки), значит, слишком маленькие чанки содержат недостаточно информации, а слишком большие — лишнюю;
- подали данные в модель без предобработки, не выбросив лишнее и не расставив приоритеты между чанками;
- не дообучили LLM, чтобы модель больше знала о нужном домене и отвечала в нужном стиле.

Получается, чтобы RAG хорошо работал, его нужно тщательно настроить. Ниже мы разберёмся, что именно надо делать.

## Как бороться с галлюцинациями в RAG?

- добавьте идентификаторы чанкам и инструкцию, чтобы LLM выводила идентификатор фрагмента, на основе которого она сгенерировала эту часть ответа; на постобработке можно их убрать, если не хотите показывать пользователю;
- уменьшите температуру и top-k (или top-p), чтобы сделать ответы более детерменированными;
- добавьте ещё одну модель, которая перепроверит, действительно ли ответ использует только информацию из чанков;
и в последнюю очередь дообучите LLM.

## Agentic RAG
...

# Graph RAG

..

В системах RAG модель отвечает только на основе найденных источников.

Retrieval Precision/Recall@k. Доля релевантных фрагментов среди вытянутых и доля найденных релевантных в целом.
Answerable Rate. Доля вопросов, где в контексте есть достаточный факт.
Faithfulness / Groundedness. Ответ опирается на контекст? Проверяют LLM-судьёй и/или строгими сопоставлениями с цитатой.
Citation Precision. Корректность ссылок (не «галлюцинированные»).
Latency и стоимость на запрос, стабильность индекса.
Хорошая практика — собирать golden-set (вопрос → кусок источника → ожидаемый ответ/цитата) из вашей базы.

**Cписок вопросов**

https://ru.scribd.com/document/870152909/Interview-Questions-on-RAG

- RAG - как учитывать картинки, таблички в контекте пайплайна?
- Как построить мультимодальный RAG?

uv run src/prod/parsing_content.py \
    --parsing_config '{
        "bucket_name": "harris.rcs.dsml",
        "root_prefix": "merchants_site_content",
        "max_retries": 3,
        "backoff_seconds": 2.0
    }'